In [1]:
import sys
sys.path.append('../src') 
import pandas as pd
import regex as re
from src.political_ads.Preprocessor import *
from src.political_ads.keyword_filter import *
from src.political_ads.zeroshot_text_preprocessing import *

In [2]:
#Code to read from our full dataset:
preprocess = Preprocessor()
df = preprocess.file_to_df('data/all_politicians_aggregated.txt')

In [45]:
#zeroshot sample
zeroshot = pd.read_csv("unique_ads_preprocessed_labeled.csv")

# apply this to the dataframe
clean_data(zeroshot)

#apply it to df
zeroshot_cleaned = zeroshot_preprocess(zeroshot)

zeroshot_cleaned = zeroshot_cleaned[['zero_shot_label','clean_text']]

/home/gugy/Documents/political-ad-api/src/political_ads/zeroshot_text_preprocessing.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['clean_text'] = dataframe['ad_creative_body'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


In [ ]:
#zeroshot labels related to climate change:
climate_zeroshot_labels = ['climate', 'environment',
       'climate, economy', 'climate, immigration',
       'environment, education', 'climate, education',
       'climate, inequality',
       'environment, economy']

zeroshot_cleaned = zeroshot_cleaned[zeroshot_cleaned['zero_shot_label'].isin(climate_zeroshot_labels) == True]

zeroshot_cleaned = zeroshot_cleaned[zeroshot_cleaned.clean_text.duplicated()==False]

In [29]:
# Adding preprocessing steps to the dataset
NaNFree = df[df.ad_creative_body.isna()==False]
clean_data(NaNFree)
full_set = zeroshot_preprocess(NaNFree)

#Adding keyword climate labels
c_filter = Filter()
c_filter.add_climate_label(full_set)


/home/gugy/Documents/political-ad-api/src/political_ads/zeroshot_text_preprocessing.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['clean_text'] = dataframe['ad_creative_body'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


Now to the analysis and comparison in performance of the two approaches...


In [93]:
full = full_set.merge(zeroshot_cleaned, left_on='clean_text', right_on='clean_text', how='left')

#full_test = full_set.merge(zeroshot_cleaned, left_on='clean_text', right_on='clean_text', how='right')

In [90]:
full[full.zero_shot_label.isna()==True]

,ad_creation_time,ad_creative_body,spend,impressions,delivery_by_region,demographic_distribution,page_id,page_name,bylines,id,spend_lo,spend_hi,impressions_lo,impressions_hi,clean_text,keyword_label,zero_shot_label


In [68]:

true_positives = full[(full.keyword_label=='Climate') & (full.zero_shot_label.isna()==False)]
false_positives = full[(full.keyword_label=='Climate') & (full.zero_shot_label.isna()==True)]
true_negatives = full[(full.keyword_label=='Non-climate') & (full.zero_shot_label.isna()==True)]
false_negatives = full[(full.keyword_label=='Non-climate') & (full.zero_shot_label.isna()==False)]

In [85]:
total_climate_zero = len(full[full.zero_shot_label.isna()==False])
total_climate_keyword = len(full[full.keyword_label=='Climate'])

In [88]:
print("zeroshot climate", total_climate_zero)
print("keyword climate", total_climate_keyword)

zeroshot climate 19331
keyword climate 19176


In [78]:
#Benchmarked against the Zeroshot.
print("True positives", len(true_positives), len(true_positives)/len(full[full.zero_shot_label.isna()==False]))
print("False positives",len(false_positives), len(false_positives)/len(full[full.zero_shot_label.isna()==False]))
print("True negatives", len(true_negatives), len(true_negatives)/len(full[full.zero_shot_label.isna()==True]))
print("False negatives", len(false_negatives), len(false_negatives)/len(full[full.zero_shot_label.isna()==False]))

True positives 18876 0.976462676529926
False positives 300 0.01551911437587295
True negatives 578428 0.9994816217635919
False negatives 455 0.023537323470073975


In [99]:
false_negatives.groupby("zero_shot_label").count()

,ad_creation_time,ad_creative_body,spend,impressions,delivery_by_region,demographic_distribution,page_id,page_name,bylines,id,spend_lo,spend_hi,impressions_lo,impressions_hi,clean_text,keyword_label
zero_shot_label,,,,,,,,,,,,,,,,
climate,16,16,16,16,15,15,16,16,16,16,16,16,16,16,16,16
"climate, economy",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
environment,430,430,430,430,318,318,430,430,430,430,430,430,430,430,430,430
"environment, economy",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
"environment, education",6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


false negatives examples

In [103]:
false_negatives.clean_text.drop_duplicates().iloc[2]

'it was my honor to present coastal bend college with 14 million in federal funding these funds will directly help students by assisting with the cost of textbooks transportation and student debt it will also help the college itself make the upgrades it needs to maintain a safe comfortable environment it is always a priority for me to support students in their pursuit of higher education '

In [116]:
unique_texts = false_negatives.clean_text.drop_duplicates()

In [117]:
for val in unique_texts.iteritems():
    print(val)

(21697, ' i believe that farms particularly family farms independent farms here in iowa can be a big part of the solution warren said we should be investing in helping those farms make sure that they are a part of cleaning up and improving our environment ')
(133492, 'the outdoors are important to our culture economy like my page stand for public lands ')
(133986, 'it was my honor to present coastal bend college with 14 million in federal funding these funds will directly help students by assisting with the cost of textbooks transportation and student debt it will also help the college itself make the upgrades it needs to maintain a safe comfortable environment it is always a priority for me to support students in their pursuit of higher education ')
(134103, 'governor abbott plans to ignore local communities and basic facts to waste tax dollars that should go to education and keep building an ineffective border wall i will defend our environment wildlife and property owners whose land